# Import packages

In [ ]:
import numpy as np # To work with arrays
import time
import matplotlib.pyplot as plt # for showing plots

# Generate a simple line without intercept

In [ ]:
x = np.random.uniform(0,100, 1000)                 #### Generate 1000 random points in range (0,100) uniformly
B = 2.1438                                         #### Arbitrary TRUE slope 
y = B * x                                          #### Calculate y values 
plt.plot(x,y, c='blue', label='best line')         #### Plot y(x)
plt.xlabel("X")
plt.ylabel("y")

y_gt = y + np.random.normal(0, 100, len(x))        #### Add Gaussian Noise to the generate a real dataset (samples)
plt.plot(x, y_gt ,'.', c='red', label='Noisy data') ### Plot red points of the samples 
plt.plot(x, -5*x, c='green', label='Initial guess') ### Plot a line of initial guess y = (-5) * x
plt.legend()
plt.show()

# MSE Loss and its gradient 

$$ loss = MSE = \frac{1}{2n} \sum_{i=1}^{n} (y_i - Bx_i)^2$$
$$ \frac{\partial L}{\partial B} = \frac{1}{n} \sum_{i=1}^{n} (-x_i)(y_i-Bx_i)$$
$$ \frac{\partial L}{\partial B} = \frac{1}{n} \sum_{i=1}^{n} x_i(Bx_i-y)$$

In [ ]:
def predict(x , B):
    return np.multiply(B , x)

def MSE(y , pred):
    return 0.5 * np.mean(np.power(y- pred , 2)) 

def gradient(x, y, B):
    return np.mean(np.multiply(x , (B*x-y)))

In [ ]:
def gradient_descent(data , init_B, num_epochs , lr):
    x = data[0]
    y = data[1]
    B = init_B
    B_list = []                    ## History of B
    loss_list = []                 ## History of Loss values
    for i in range(num_epochs):
        B_list.append(B) ## Adds the current B to the list
        ## 1) Compute Loss
        pred = predict(x, B)
        loss = MSE(y, pred) 
        loss_list.append(loss)
        ## 2) Compute Gradient
        grad = gradient(x, y, B)
        ## 3) Update the Model's Parameter
        B = B - lr * grad
        
    B_list.append(B)        ## Save the final Coefficient
    pred = predict(x, B) 
    loss = MSE(y , pred)
    loss_list.append(loss)  ## Save the final loss value
    
    return B_list , loss_list

In [ ]:
## DATA SHAPE 
print("Shape of x is: ", x.shape)
print("Shape of y is: ", y_gt.shape)
data = np.vstack((x ,y_gt))
print("Shape of data is: ", data.shape)

### Solve Linear Regression with Gradient Descent 

In [ ]:
B, loss = gradient_descent(data , init_B=-5 , num_epochs=20, lr=0.0005) 
print("final B is: ", B[-1])
print("final Loss is: ", loss[-1])


plt.plot(loss)
plt.xlabel("Epoch = Iteration")
plt.ylabel("Loss")
plt.title("Loss vs. Iteration")
plt.show()

plt.plot(B)
plt.xlabel("Epoch = Iteration")
plt.ylabel("B")
plt.title("B vs. Iteration")
plt.show()

## Effect of noise magnitude (data distribution)

In [ ]:
y_gt = y + np.random.normal(0, 100, len(x))
plt.plot(x, y_gt,'.',c='red', label=r'$\sigma = 100$')

y_gt = y + np.random.normal(0, 10, len(x))
plt.plot(x, y_gt,'.',c='orange', label=r'$\sigma = 10$')
plt.plot(x,y ,c='blue', label='True Line')
plt.legend()
plt.show()

In [ ]:
print("True B coefficient is : ", 2.1438)
for noise in [0.001 , 1 , 10, 100, 1000, 10000]:
    y_gt = y + np.random.normal(0, noise, len(x))
    data = np.vstack((x ,y_gt))
    B, loss = gradient_descent(data , -5 , 50, 0.0001)
    print(f"final B for noise {noise} is: ", round(B[-1],5)) 

## Effect of Learning rate

In [ ]:
y_gt = y + np.random.normal(0, 10, len(x))
data = np.vstack((x ,y_gt))
    
for lr in [1e-6,0.00001 , 0.0001 , 0.0005, 0.001]:
    B, loss = gradient_descent(data , -5 , 100, lr )
    print(f"final B for learning rate {lr} is: ", round(B[-1],5))
    
    plt.figure(figsize=(15,4))
    ax1 = plt.subplot(1,2, 1)
    ax1.plot(loss)
    ax1.set_xlabel("epoch")
    ax1.set_ylabel("loss")
    plt.title(f"LR is {lr}")
    
    ax2 = plt.subplot(1,2, 2)
    ax2.plot(B)
    ax2.set_xlabel("epoch")
    ax2.set_ylabel("B")
    
    plt.title(f"LR is {lr}")
    plt.show()


## Loss Landscape

In [ ]:
def plot_landscape():
    y_gt = y + np.random.normal(0, 10, len(x))
    B_arr = np.linspace(-6, 10 , 100)
    losses = []
    for B in B_arr:
        pred = predict(x, B)
        loss = MSE(y_gt, pred)
        losses.append(loss)

    plt.plot(B_arr, losses)
    plt.title("Loss landscape")
    plt.xlabel("B")
    plt.ylabel("loss")
    
plot_landscape()

In [ ]:
def plot_traj(data, lr, num_epoch, init_B):
    B, loss = gradient_descent(data , init_B , num_epoch, lr)
    print("final B is: ", B[-1])
    plot_landscape()
    for i in range(len(B)-1):
        plt.scatter(B[i], loss[i], c='red')
        plt.plot([B[i], B[i+1]], [loss[i], loss[i+1]], linestyle='dashed', c='grey')
    plt.show()
    
data = np.vstack((x ,y_gt))

plot_traj(data, 0.00001, 10, -5)

plot_traj(data, 0.0001, 10, -5)

plot_traj(data, 0.0005, 10, -5)

plot_traj(data, 0.00063, 3, -5)



## Momentum Gradient Descent

In [ ]:
def momentum_gd(data , init_B, num_epochs , lr):
    x = data[0]
    y = data[1]
    B = init_B
    B_list = []
    loss_list = []
    v = 0
    beta = 0.9
    for i in range(num_epochs):
        B_list.append(B) ## Adds the current B to the list
        ## 1) Compute loss
        pred = predict(x, B)
        loss = MSE(y, pred)
        loss_list.append(loss)
        ## 2) Compute Gradient
        grad = gradient(x, y, B)
        ## 3) Update with Momentum
        v = beta*v + (1-beta)*(grad)
        v_corr = v / (1-beta**(i+1) )
        B = B - lr * v_corr
        
    B_list.append(B)
    pred = predict(x, B)
    loss = MSE(y, pred)
    loss_list.append(loss)
    
    return B_list , loss_list

In [ ]:
def plot_traj(data, lr, num_epoch, init_B):
    B, loss = momentum_gd(data , init_B , num_epoch, lr)
    print("final B is: ", B[-1])
    plot_landscape()
    for i in range(len(B)-1):
        plt.scatter(B[i], loss[i], c='red')
        plt.plot([B[i], B[i+1]], [loss[i], loss[i+1]], linestyle='dashed', c='grey')
    plt.show()
    
data = np.vstack((x ,y_gt))

plot_traj(data, 0.00001, 10, -5)

plot_traj(data, 0.0001, 100, -5)

plot_traj(data, 0.0005, 10, -5)

plot_traj(data, 0.00063, 100, -5)



# Batch, SGD, Mini-Batch

### Iteration vs. Epoch
Iteration is about updating the model's weights and coefficients (Parameters). Each time we update the parameters, we call it an iteration. 
Epoch is about the dataset. Each time we have used all the samples in the dataset to update our model, we call it an epoch. 
Therefore, if we use full batch gradient descent, an iteration is equivalent to an epoch. 
On the other hand, for Stochastic GD, we update the model after visiting each sample. So, after N iterations where N is the dataset size, we have performed an epoch. 

## Batch

In [ ]:
def gradient_descent(data , init_B, num_epochs , lr):
    x = data[0]
    y = data[1]
    B = init_B
    B_list = []
    loss_list = []
    for i in range(num_epochs):
        B_list.append(B) ## Adds the current B to the list
        pred = predict(x , B)
        loss = MSE(y , pred)
        loss_list.append(loss)
        grad = gradient(x , y , B)
        B = B - lr * grad
        
    B_list.append(B)
    pred = predict(x , B)
    loss = MSE(y , pred)
    loss_list.append(loss)
    
    return B_list , loss_list

In [ ]:
x = np.random.uniform(0,100, 1000)
B = 2.1438
y = B * x
y_gt = y + np.random.normal(0, 100, len(x))
data = np.vstack((x ,y_gt))
B, loss = gradient_descent(data , -5 , 10, 0.0001)
print("Final B after 10 epochs is: ", B[-1])
plt.plot(loss)
plt.xlabel("Iteration = Epoch")
plt.ylabel("Loss")
plt.show()

# SGD

In [ ]:
def SGD(data , init_B, num_iter , lr):
    x = data[0]
    y = data[1]
    B = init_B
    B_list = []
    loss_list = []
    for i in range(num_iter):
        j = i % len(x)
        B_list.append(B) ## Adds the current B to the list
        loss = 0.5 * np.power(y[j]- B*x[j] , 2) 
        loss_list.append(loss)
        grad = np.multiply(x[j] , (B*x[j]-y[j])) 
        B = B - lr * grad
        
    B_list.append(B)
    loss = 0.5 * np.mean(np.power(y- B*x , 2))
    loss_list.append(loss)
    
    return B_list , loss_list

In [ ]:
B, loss = SGD(data , -5 , 10000, 0.000001)
print("Final B after 10000 iterations = 10 epoch is: ", B[-1])
plt.plot(loss)
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()

## Mini-Batch GD

In [ ]:
def Mini_GD(data , init_B, num_iter , lr, batch_size):
    x = data[0]
    y = data[1]
    B = init_B
    B_list = []
    loss_list = []
    num_batches = len(x) // batch_size
    num_epochs = num_iter // num_batches 
    for epoch in range(num_epochs):
        for b in range(num_batches):
            B_list.append(B) ## Adds the current B to the list
            loss = 0.5 * np.mean(np.power(y[b*batch_size: (b+1)*batch_size]- B*x[b*batch_size: (b+1)*batch_size] , 2)) 
            loss_list.append(loss)
            grad = np.mean(np.multiply(x[b*batch_size: (b+1)*batch_size] , (B*x[b*batch_size: (b+1)*batch_size]-y[b*batch_size: (b+1)*batch_size]))) 
            B = B - lr * grad
        
    B_list.append(B)
    loss = 0.5 * np.mean(np.power(y- B*x , 2))
    loss_list.append(loss)
    
    return B_list , loss_list

In [ ]:
B, loss = Mini_GD(data , -5 , 100, 0.0001, 100)
print("Final B after 100 iterations = 10 epoch is: ", B[-1])
plt.plot(loss)
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()

## Loss Landscape for SGD

In [ ]:
y_gt = y + np.random.normal(0, 30, len(x))
B_arr = np.linspace(1, 4 , 100)

loss_arr = np.zeros((4, len(B_arr)))
for t in range(len(B_arr)):
    loss_arr[0][t]  = 0.5 * np.power(y_gt[0]-B_arr[t]*x[0] , 2)
    loss_arr[1][t]  = 0.5 * np.power(y_gt[1]-B_arr[t]*x[1] , 2)
    loss_arr[2][t]  = 0.5 * np.power(y_gt[2]-B_arr[t]*x[2] , 2)
    
    loss_arr[3][t] = 0.5 * np.mean(np.power(y_gt-B_arr[t]*x , 2))
    
    
    
plt.plot(B_arr, loss_arr[0], label='Sample 1')
plt.plot(B_arr, loss_arr[1], label='Sample 2')
plt.plot(B_arr, loss_arr[2], label='Sample 3')
plt.plot(B_arr, loss_arr[3], label='All Samples')
plt.legend()
plt.title("Loss landscape")
plt.xlabel("B")
plt.ylabel("loss")
plt.show()

## True or False
### Using SGD we don't have the real loss landscape, but we estimate it.  (T or F) TRUE
### But using Batch GD, we have the real loss landscape, and have the exact gradient. (T or F) FALSE

# References

A nice visualization of GD, SGD and Mini-Batch: http://www.deeplearning.ai/ai-notes/optimization/